In [1]:
import re
import json

In [3]:
def source(document):
    with open(f"NIKL_NEWSPAPER(v1.0)/{document[:14]}.json","rb") as f:
        file=json.load(f)
    for x in file["document"]:
        if x["id"]==document:
            return sum([".<q>".join(y["form"].split(". ")).split("<q>") for y in x["paragraph"] if y["form"][-1]=="."],[])

In [4]:
with open("NIKL_SC.json","rb") as f:
    sum_file=json.load(f)
corpus1=[{"src":source(x["document_id"]),"tgt":x["topic_sentences"]} for x in sum_file["data"]]

In [6]:
def load_jsonl(input_path):
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data
train=load_jsonl("train.jsonl")

Loaded 42803 records from train.jsonl


In [10]:
corpus2=[{"src":x["article_original"],"tgt":[x["article_original"][i] for i in x["extractive"]]} for x in train]

In [11]:
corpus=corpus1+corpus2

In [13]:
corpus[0]

{'src': ['여야가 6일 임시국회 종료 이틀을 남기고 주요 쟁점법안 처리 문제를 일괄 타결했다.',
  '이로써 지난해 12월 10일 개회 이래 4주 동안 야당의 본회의장 불법 점거와 국회의장의 질서유지권 발동 등으로 가파르게 대치했던 정국이 정상화됐다.',
  '한나라당 홍준표, 민주당 원혜영, 선진과 창조의 모임 문국현 원내대표는 이날 국회 귀빈식당에서 원내대표 회담을 두 차례 열어 미디어 관계 법안 등 쟁점 법안의 처리 방안에 대한 10개 항에 합의했다.',
  '여야는 이견이 없는 민생법안 등 100여 건을 이번 임시국회에서 협의 처리하되 아직 상임위원회에 상정되지 않았거나 심의를 거치지 못한 법안은 9일 다시 임시국회를 열어 처리하기로 했다.',
  '그러나 여야는 이날 합의에서 상당수 쟁점법안의 처리 시기는 물론이고 법안 상정 여부조차 정하지 못해 2월 1일 열리는 임시국회에서 이를 둘러싼 ‘제2차 입법전쟁’이 벌어질 가능성이 높다.',
  '여야는 최대 쟁점이었던 미디어 관계 법안 8건 중 언론중재법 등 여야 간 이견이 없는 법안 2건만 이번 임시국회에서 협의 처리하고 신문·방송 겸영 허용과 대기업 방송 진출 허용 등 여야가 맞서는 6개 법안은 이른 시일 안에 합의 처리하도록 노력하기로 했다.',
  '한미 자유무역협정(FTA) 비준동의안은 미국 차기 행정부가 출범(이달 20일)한 뒤 이른 시일 안에 협의 처리하기로 했다.',
  '출자총액제한제도 폐지 법안은 이번 임시국회에서 상임위원회에 상정하되 2월 임시국회에서 협의 처리하고 금산분리 완화 법안 역시 이번 임시국회에서 상정하되 여야가 합의 처리하도록 노력하기로 했다.',
  '재외국민에게 대선과 총선의 투표권을 부여하는 내용의 공직선거법 개정안은 여야 동수로 정치개혁특별위원회를 구성해 2월 임시국회에서 합의 처리하기로 했다.',
  '이에 앞서 민주당은 이날 오전 국회 본회의장과 행정안전위, 정무위 회의장 점거 농성을 풀었다.',
  '민주당은 7일엔 문화체육관광방송통신위 회의장에서 철수

In [2]:
def clean(x):
    return x.replace("(","").replace(")","").replace("{","").replace("}","").replace("[","").replace("]","").replace('"',"").replace("'","").replace("=","")

In [9]:
from konlpy.tag import Okt
def load_data(article):
    okt=Okt()
    if (article["src"]) and (article["tgt"]):
        src_txt=article["src"]
        tgt_txt=article["tgt"]

        source=[okt.morphs(clean(sen)) for sen in src_txt]
        tgt=[okt.morphs(clean(sen)) for sen in tgt_txt]
    
        return {'src': source, 'tgt': tgt}

def format_to_lines(corpus):
    ls=[]
    for x in corpus:
        if load_data(x) is not None:
            ls.append(load_data(x))
    with open(f"cnndm_sample.train.0.json","w") as file:
        json.dump(ls,file)

format_to_lines(corpus)